<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/PyTorch/Basic/ProtBert-BFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtBert-BFD pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 778kB 8.0MB/s 
     |████████████████████████████████| 1.1MB 13.0MB/s 
     |████████████████████████████████| 3.0MB 45.9MB/s 
     |████████████████████████████████| 890kB 64.5MB/s 


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm

<b>2. Load the vocabulary and ProtBert-BFD Model<b>

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False )

In [4]:
model = AutoModel.from_pretrained("Rostlab/prot_bert_bfd")

<b>3. Load the model into the GPU if avilabile<b>

In [5]:
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer,device=0 )

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [6]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [7]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>5. Extracting sequences' features and covert the output to numpy if needed<b>

In [8]:
embedding = fe(sequences_Example)

In [9]:
embedding = np.array(embedding)

In [10]:
print(embedding)

[[[ 0.07873335  0.02530318  0.10022405 ... -0.11921909 -0.08556543
    0.0157964 ]
  [ 0.05551133 -0.10461304 -0.03253962 ...  0.05091606  0.04318975
    0.10181108]
  [ 0.13895561 -0.046583    0.02193631 ...  0.06942613  0.14762992
    0.06503808]
  ...
  [ 0.08129995 -0.1092955  -0.03022903 ...  0.08717731  0.02061446
    0.05156654]
  [ 0.06197417 -0.06417818 -0.02039655 ... -0.02796507  0.0884005
    0.07532689]
  [ 0.09792715 -0.02395107 -0.07244891 ... -0.02299493  0.03639549
    0.0507541 ]]

 [[ 0.11757391 -0.01604314  0.12586999 ... -0.0931114  -0.09125263
   -0.01659234]
  [-0.06304268 -0.23687428 -0.07115868 ... -0.03852162 -0.00322069
   -0.05244054]
  [ 0.01905588 -0.105173   -0.02930211 ... -0.00238627 -0.09289714
    0.02722595]
  ...
  [-0.00986713 -0.15825894 -0.08714887 ... -0.10954074 -0.03796846
   -0.03043361]
  [-0.01160221 -0.16065687 -0.11397075 ... -0.07776542 -0.00063999
   -0.02496129]
  [ 0.002604   -0.1685513  -0.06161089 ... -0.09357076 -0.0614622
   -0.02

<b>Optional: Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtBert-BFD model<b>

In [11]:
features = [] 

for seq_num in range(len(embedding)):
    seq_len = len(sequences_Example[seq_num].replace(" ", ""))
    start_Idx = 1
    end_Idx = seq_len+1
    seq_emd = embedding[seq_num][start_Idx:end_Idx]
    features.append(seq_emd)

In [12]:
print(features)

[array([[ 0.05551133, -0.10461304, -0.03253962, ...,  0.05091606,
         0.04318975,  0.10181108],
       [ 0.13895561, -0.046583  ,  0.02193631, ...,  0.06942613,
         0.14762992,  0.06503808],
       [ 0.14610603, -0.08092842, -0.12500416, ..., -0.03651231,
         0.02485525,  0.07977536],
       ...,
       [ 0.02349902, -0.01549769, -0.05685329, ..., -0.01342281,
         0.01704315,  0.06431052],
       [ 0.08129995, -0.1092955 , -0.03022903, ...,  0.08717731,
         0.02061446,  0.05156654],
       [ 0.06197417, -0.06417818, -0.02039655, ..., -0.02796507,
         0.0884005 ,  0.07532689]]), array([[-0.06304268, -0.23687428, -0.07115868, ..., -0.03852162,
        -0.00322069, -0.05244054],
       [ 0.01905588, -0.105173  , -0.02930211, ..., -0.00238627,
        -0.09289714,  0.02722595],
       [ 0.07721861, -0.1703198 , -0.13987812, ..., -0.08390203,
         0.03587941, -0.01317161],
       [ 0.00872737, -0.1771819 , -0.05856298, ..., -0.09918059,
        -0.06392955,